# II. Data Exploration
Once the tables have been cleaned, you can see repeated values and associations to which you need to assign a specific table.

1. Import libraries and scr.py file
2. [Data Exploration](#import-clean-documents)
    - 2.1. [special_features](#import-special_features)
    - 2.2. [category](#import-category)
    - 2.3. [films & actors](#import-films-&-actors)
    - 2.4. [inventory & rental](#import-inventory-&-rental)
    - 2.4. [language](#import-language)
4. [Export documents](#export-documents)

## 1.Import libraries and scr.py file

In [61]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [62]:
import scr as sc #Functions

In [63]:
import pylab as plt  
import seaborn as sns

%matplotlib inline

## 2. Data Exploration

###  2.1. special_features

#### Create a new table special_features

In [10]:
film = pd.read_csv('../data/1-clean/film.csv')
film.head(3)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features
0,1,ACADEMY DINOSAUR,A EPIC DRAMA OF A FEMINIST AND A MAD SCIENTIST...,2006,1,0,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes"
1,2,ACE GOLDFINGER,A ASTOUNDING EPISTLE OF A DATABASE ADMINISTRAT...,2006,1,0,3,4.99,48,12.99,G,"Trailers,Deleted Scenes"
2,3,ADAPTATION HOLES,A ASTOUNDING REFLECTION OF A LUMBERJACK AND A ...,2006,1,0,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes"


First, we call a function that receives the column to return a list with all the elements and transform them to set. Then we create a DataFrame with the values of the list and its own id.

In [11]:
sf_set= set(sc.column_to_list(film.special_features))

sf_set.add('None') #For the films that doesn't have one

sf_set

{'Behind the Scenes', 'Commentaries', 'Deleted Scenes', 'None', 'Trailers'}

In [12]:
sf_dict= {'sp_id': [i for i in range(0,len(sf_set))], 
          'special_feature': list(sf_set)}

special_features= pd.DataFrame(sf_dict)

special_features

,sp_id,special_feature
0,0,None
1,1,Commentaries
2,2,Trailers
3,3,Deleted Scenes
4,4,Behind the Scenes


#### Asociación con film: many to many
On the other hand, we add 4 columns to the df film with its special_features separated. With the previos df we know the maximum is 4, so we create a function that receives the string of all the special_features per film and separates it (with the character that we tell it), in a list of the length that we want, in this case 4. And add it

In [13]:
sc.split_string('Deleted Scenes,Behind the Scenes', ',', 4, 'None')

['Deleted Scenes', 'Behind the Scenes', 'None', 'None']

In [14]:
film[['sf1','sf2','sf3','sf4']]= [sc.split_string(e, ',',4,'None') for e in film.special_features]

film.head(2)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,sf1,sf2,sf3,sf4
0,1,ACADEMY DINOSAUR,A EPIC DRAMA OF A FEMINIST AND A MAD SCIENTIST...,2006,1,0,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",Deleted Scenes,Behind the Scenes,None,None
1,2,ACE GOLDFINGER,A ASTOUNDING EPISTLE OF A DATABASE ADMINISTRAT...,2006,1,0,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",Trailers,Deleted Scenes,None,None


And we take the film_id and the four new columns and create a temporary dataframe. The intention of all this is to create a table "film_id_has_special_features" in SQL

In [15]:
film_sf_temp= film[['film_id','sf1','sf2','sf3','sf4']]

film_sf_temp.head(5)

,film_id,sf1,sf2,sf3,sf4
0,1,Deleted Scenes,Behind the Scenes,None,None
1,2,Trailers,Deleted Scenes,None,None
2,3,Trailers,Deleted Scenes,None,None
3,4,Commentaries,Behind the Scenes,None,None
4,5,Deleted Scenes,None,None,None


In [16]:
lst_films=[]
lst_sp=[]

for index, row in film_sf_temp.iterrows():
    
    film_id= row['film_id']
    
    for col in ['sf1', 'sf2', 'sf3', 'sf4']:
        
        value = row[col]
        
        if(value!='None'):
            
            lst_films.append(film_id)
            
            lst_sp.append(value)
            
#If you are reading this, is because I couldn't find the time to redo this. It is not the efficienter way and I know :(             

In [17]:
dict_temp= {'film_id': lst_films, 
          'sp_id': lst_sp}

film_has_special_features = pd.DataFrame(dict_temp)

film_has_special_features.head(3)

,film_id,sp_id
0,1,Deleted Scenes
1,1,Behind the Scenes
2,2,Trailers


Convert the special_features column to its corresponding id

In [18]:
special_features.head()

,sp_id,special_feature
0,0,None
1,1,Commentaries
2,2,Trailers
3,3,Deleted Scenes
4,4,Behind the Scenes


#### Create table film_has_special_features

In [19]:
for e in range(len(film_has_special_features)):
    
    for index, row in special_features.iterrows():
        
        if(film_has_special_features.loc[e, 'sp_id']==row['special_feature']):
            
                film_has_special_features.loc[e, 'sp_id']=row['sp_id']
    

In [20]:
film_has_special_features.head()

,film_id,sp_id
0,1,3
1,1,4
2,2,2
3,2,3
4,3,2


Finally, we delete the related columns of the table film

In [21]:
film.drop(columns=['special_features','sf1','sf2','sf3','sf4'], inplace=True)

### 2.2. category
The only table that associates category with "film" information is "old_HDD", so I created a new column in "films" called "category_id" and filled it with the records that match in "old_HDD.title" with "category_id". 
The rest, with "Unknown" category

In [22]:
old_HDD = pd.read_csv('../data/1-clean/old_HDD.csv')

old_HDD.head()

,first_name,last_name,title,category_id
0,Penelope,Guiness,ACADEMY DINOSAUR,6
1,Penelope,Guiness,ANACONDA CONFESSIONS,2
2,Penelope,Guiness,ANGELS LIFE,13
3,Penelope,Guiness,BULWORTH COMMANDMENTS,10
4,Penelope,Guiness,CHEAPER CLYDE,14


In [23]:
film.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating
0,1,ACADEMY DINOSAUR,A EPIC DRAMA OF A FEMINIST AND A MAD SCIENTIST...,2006,1,0,6,0.99,86,20.99,PG
1,2,ACE GOLDFINGER,A ASTOUNDING EPISTLE OF A DATABASE ADMINISTRAT...,2006,1,0,3,4.99,48,12.99,G
2,3,ADAPTATION HOLES,A ASTOUNDING REFLECTION OF A LUMBERJACK AND A ...,2006,1,0,7,2.99,50,18.99,NC-17
3,4,AFFAIR PREJUDICE,A FANCIFUL DOCUMENTARY OF A FRISBEE AND A LUMB...,2006,1,0,5,2.99,117,26.99,G
4,5,AFRICAN EGG,A FAST-PACED DOCUMENTARY OF A PASTRY CHEF AND ...,2006,1,0,6,2.99,130,22.99,G


In [24]:
film['category_id'] = np.nan

In [25]:
film = film.apply(sc.fill_column, df_to_compare=old_HDD, column_to_compare='title', column_to_change='category_id', axis=1)

Once the column is filled, there will be null values that we will replace with category_id 17 that we add to the category table directly. Then we change the value to integer which is how it will go in the database

In [26]:
film[(film.category_id.isna())].shape

(386, 12)

In [27]:
category = pd.read_csv('../data/1-clean/category.csv')

category.head()

,category_id,name
0,1,Action
1,2,Animation
2,3,Children
3,4,Classics
4,5,Comedy


In [28]:
new_row = {'category_id': 17,'name': 'Unassigned'}

category.loc[len(category)] = new_row

category.tail(1)

,category_id,name
16,17,Unassigned


In [29]:
film.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   film_id               1000 non-null   int64  
 1   title                 1000 non-null   object 
 2   description           1000 non-null   object 
 3   release_year          1000 non-null   int64  
 4   language_id           1000 non-null   int64  
 5   original_language_id  1000 non-null   int64  
 6   rental_duration       1000 non-null   int64  
 7   rental_rate           1000 non-null   float64
 8   length                1000 non-null   int64  
 9   replacement_cost      1000 non-null   float64
 10  rating                1000 non-null   object 
 11  category_id           614 non-null    float64
dtypes: float64(3), int64(6), object(3)
memory usage: 93.9+ KB


In [30]:
film['category_id']= film['category_id'].fillna(17)

In [31]:
film['category_id']=film['category_id'].astype('int')

Last but no least, drop the category_id column from old_HDD

In [32]:
old_HDD.drop(columns= 'category_id', inplace=True)

### 2.3. films & actors
Now, let's get the party started: 
Actor and film is a many to many relation. So, the only way to associate those two is to create the "film_has_actor" table with their corresponding id. That is only possible thanks to the charming old_HDD table.

First step, add film_id to old_HDD

In [33]:
old_HDD['film_id']=0

In [34]:
old_HDD = old_HDD.apply(sc.fill_column, df_to_compare=film, column_to_compare='title', column_to_change='film_id', axis=1)

old_HDD.head()

,first_name,last_name,title,film_id
0,Penelope,Guiness,ACADEMY DINOSAUR,1
1,Penelope,Guiness,ANACONDA CONFESSIONS,23
2,Penelope,Guiness,ANGELS LIFE,25
3,Penelope,Guiness,BULWORTH COMMANDMENTS,106
4,Penelope,Guiness,CHEAPER CLYDE,140


Second, add actor_id. For it we need to concat first and last name to compare it in the function

In [35]:
old_HDD['full_name']= old_HDD['first_name']+' '+old_HDD['last_name']

In [36]:
old_HDD.head()

,first_name,last_name,title,film_id,full_name
0,Penelope,Guiness,ACADEMY DINOSAUR,1,Penelope Guiness
1,Penelope,Guiness,ANACONDA CONFESSIONS,23,Penelope Guiness
2,Penelope,Guiness,ANGELS LIFE,25,Penelope Guiness
3,Penelope,Guiness,BULWORTH COMMANDMENTS,106,Penelope Guiness
4,Penelope,Guiness,CHEAPER CLYDE,140,Penelope Guiness


In [37]:
actors = pd.read_csv('../data/1-clean/actor.csv')
actors.head(2)

,actor_id,first_name,last_name
0,1,Penelope,Guiness
1,2,Nick,Wahlberg


In [38]:
actors['full_name']= actors['first_name']+' '+actors['last_name']

Now we can add actor_id to old_HDD

In [39]:
old_HDD['actor_id']= ''

In [40]:
old_HDD = old_HDD.apply(sc.fill_column, df_to_compare=actors, column_to_compare='full_name', column_to_change='actor_id', axis=1)

old_HDD.head()

,first_name,last_name,title,film_id,full_name,actor_id
0,Penelope,Guiness,ACADEMY DINOSAUR,1,Penelope Guiness,1
1,Penelope,Guiness,ANACONDA CONFESSIONS,23,Penelope Guiness,1
2,Penelope,Guiness,ANGELS LIFE,25,Penelope Guiness,1
3,Penelope,Guiness,BULWORTH COMMANDMENTS,106,Penelope Guiness,1
4,Penelope,Guiness,CHEAPER CLYDE,140,Penelope Guiness,1


In [41]:
old_HDD.drop(columns=['first_name','last_name','title','full_name'], inplace=True)

old_HDD.head(2)

,film_id,actor_id
0,1,1
1,23,1


In [42]:
actors.drop(columns=['full_name'], inplace=True)

actors.head(2)

,actor_id,first_name,last_name
0,1,Penelope,Guiness
1,2,Nick,Wahlberg


### 2.4. inventory & rental

The way both tables are set up, it only allows the rental of one film at a time. So the best idea is to reconvert that table to rental_detail which will be the intermediate table of the many-to-many relationship between rental and film and it will only have: film_id and rental_id 

In [43]:
inventory = pd.read_csv('../data/1-clean/inventory.csv')

inventory.head()

,inventory_id,film_id,store_id
0,1,1,1
1,2,1,1
2,3,1,1
3,4,1,1
4,5,1,2


In [44]:
rental = pd.read_csv('../data/1-clean/rental.csv')

rental.tail()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id
995,996,2005-05-31 00:08:25,4243,216,2005-06-02 00:17:25,2
996,997,2005-05-31 00:16:57,3395,389,2005-06-01 22:41:57,1
997,998,2005-05-31 00:25:10,4433,413,2005-06-03 06:05:10,2
998,999,2005-05-31 00:25:56,1774,332,2005-06-08 19:42:56,2
999,1000,2005-05-31 00:46:31,1498,64,2005-06-06 06:14:31,2


Drop store_id because the plan is an initial store and in case more are opened, just create a store table and associate with rental

In [45]:
inventory.drop('store_id', axis=1, inplace=True)

inventory.head()

,inventory_id,film_id
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [46]:
inventory.rename(columns={"inventory_id": "rental_id"}, inplace=True)

inventory.head(2)

,rental_id,film_id
0,1,1
1,2,1


### 2.5 language
As last step, we add 'Unknown' to "language" and we add the unknown language id to "original_language_id" in "film"

###### Add 0 to language DataFrame as "unknown"

In [47]:
language = pd.read_csv('../data/1-clean/language.csv')

language.head(2)

,language_id,name
0,1,English
1,2,Italian


In [48]:
new_row = {'language_id': 7,'name': 'Unknown'}

language.loc[len(language)] = new_row

language.tail(1)

,language_id,name
6,7,Unknown


In [49]:
film.original_language_id= 7

In [50]:
film.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,category_id
0,1,ACADEMY DINOSAUR,A EPIC DRAMA OF A FEMINIST AND A MAD SCIENTIST...,2006,1,7,6,0.99,86,20.99,PG,6
1,2,ACE GOLDFINGER,A ASTOUNDING EPISTLE OF A DATABASE ADMINISTRAT...,2006,1,7,3,4.99,48,12.99,G,11
2,3,ADAPTATION HOLES,A ASTOUNDING REFLECTION OF A LUMBERJACK AND A ...,2006,1,7,7,2.99,50,18.99,NC-17,6
3,4,AFFAIR PREJUDICE,A FANCIFUL DOCUMENTARY OF A FRISBEE AND A LUMB...,2006,1,7,5,2.99,117,26.99,G,17
4,5,AFRICAN EGG,A FAST-PACED DOCUMENTARY OF A PASTRY CHEF AND ...,2006,1,7,6,2.99,130,22.99,G,17


## 3. Export documents

In [51]:
actors.to_csv('../data/2-processed/actor.csv', index=False)

In [52]:
category.to_csv('../data/2-processed/category.csv', index=False)

In [53]:
film.to_csv('../data/2-processed/film.csv', index=False)

In [54]:
film_has_special_features.to_csv('../data/2-processed/film_has_special_features.csv', index=False)

In [55]:
language.to_csv('../data/2-processed/language.csv', index=False)

In [56]:
old_HDD.to_csv('../data/2-processed/film_has_actor.csv', index=False)

In [57]:
special_features.to_csv('../data/2-processed/special_features.csv', index=False)

In [58]:
inventory.to_csv('../data/2-processed/rental_detail.csv', index=False)

In [59]:
rental.to_csv('../data/2-processed/rental.csv', index=False)